In [1]:
import numpy as np
import pandas as pd

from mpmath import besseljzero
from scipy.special import gamma
from scipy.special import jv, iv
from scipy.optimize import minimize
from scipy.interpolate import interp1d

from scipy.stats import pearsonr
from sklearn.metrics import r2_score

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def simulate_HSDM_2D(a, mu, ndt, sigma=1, dt=0.001):
    x = np.zeros(mu.shape)
    
    rt = 0
    
    while np.linalg.norm(x, 2) < a(rt):
        x += mu*dt + sigma*np.sqrt(dt)*np.random.normal(0, 1, mu.shape)
        rt += dt
    
    theta = np.arctan2(x[1], x[0])   
    
    return ndt+rt, theta

In [3]:
def series_bessel_fpt(t, a=1, sigma=1, nu=0, n=100):
    zeros = np.asarray([float(besseljzero(0, i+1)) for i in range(n)])
    fpt = np.zeros(t.shape)
    
    for i in range(t.shape[0]):
        series = np.sum((zeros**(nu+1)/jv(nu+1, zeros)) * np.exp(-(zeros**2 * sigma**2)/(2*a**2)*t[i]))
        fpt[i] = sigma**2/(2**nu * a**2 * gamma(nu + 1)) * series
        
    return interp1d(t, fpt)

In [4]:
def HSDM_2D_likelihood(prms, RT, Theta, N_series):
    a =  prms[0]
    ndt = prms[1]
    mu = np.array([prms[2], prms[3]])
    
    tt = np.arange(0.001, max(RT)+0.02, 0.02)
    fpt = series_bessel_fpt(tt, a, sigma=1, nu=(mu.shape[0]-2)/2, n=N_series)
    
    log_lik = 0
    for i in range(len(RT)):
        rt, theta = RT[i], Theta[i]
        if rt - ndt > 0.001:
            mu_dot_x0 = mu[0]*np.cos(theta)
            mu_dot_x1 = mu[1]*np.sin(theta)
            term1 = prms[0] * (mu_dot_x0 + mu_dot_x1)
            term2 = 0.5 * np.linalg.norm(mu, 2)**2 * (rt - ndt)
            
            density = np.exp(term1 - term2) * fpt(rt - ndt)
            
            if 0.1**14 < density:
                log_lik += -np.log(density)
            else:
                log_lik += -np.log(0.1**14)
        else:
            log_lik += -np.log(0.1**14)
        
    return log_lik

In [5]:
recovery_df = {'threshold_true': [],
               'threshold_estimate': [],
               'ndt_true': [],
               'ndt_estimate': [],
               'mu1_true': [],
               'mu1_estimate': [],
               'mu2_true': [],
               'mu2_estimate': []}

min_threshold = 0.5
max_threshold = 5

min_ndt = 0.1
max_ndt = 1

min_mu = -3.5
max_mu = 3.5

N_series = 250

In [6]:
for n in range(150):
    threshold = np.random.uniform(min_threshold, max_threshold)
    a = lambda t: threshold
    ndt = np.random.uniform(min_ndt, max_ndt)
    mu = np.array([np.random.uniform(min_mu, max_mu), 
                   np.random.uniform(min_mu, max_mu)])
    
    
    recovery_df['threshold_true'].append(threshold)
    recovery_df['ndt_true'].append(ndt)
    recovery_df['mu1_true'].append(mu[0])
    recovery_df['mu2_true'].append(mu[1])
    
    RT = []
    Theta = []
    
    print(n+1, end=': ')
    for i in range(250):
        rt, theta = simulate_HSDM_2D(a, mu, ndt)
        RT.append(rt)
        Theta.append(theta)
    
    print('Simulation is done!', end=' / ')
    
    min_ans = minimize(HSDM_2D_likelihood,
                       args=(RT, Theta, N_series), 
                       x0=np.array([np.random.uniform(min_threshold, max_threshold),
                                    np.random.uniform(min_ndt, max_ndt), 
                                    np.random.uniform(min_mu, max_mu),
                                    np.random.uniform(min_mu, max_mu)]),
                       method='Nelder-Mead', 
                       bounds=[(min_threshold, max_threshold), (min_ndt, max_ndt),
                               (min_mu, max_mu), (min_mu, max_mu)])
    
    recovery_df['threshold_estimate'].append(min_ans.x[0])
    recovery_df['ndt_estimate'].append(min_ans.x[1])
    recovery_df['mu1_estimate'].append(min_ans.x[2])
    recovery_df['mu2_estimate'].append(min_ans.x[3])
    
    print('Estimation is done!')
    
recovery_df = pd.DataFrame(recovery_df)

1: Simulation is done! / Estimation is done!
2: Simulation is done! / Estimation is done!
3: Simulation is done! / Estimation is done!
4: Simulation is done! / Estimation is done!
5: Simulation is done! / Estimation is done!
6: Simulation is done! / Estimation is done!
7: Simulation is done! / Estimation is done!
8: Simulation is done! / Estimation is done!
9: Simulation is done! / Estimation is done!
10: Simulation is done! / Estimation is done!
11: Simulation is done! / Estimation is done!
12: Simulation is done! / Estimation is done!
13: Simulation is done! / Estimation is done!
14: Simulation is done! / Estimation is done!
15: Simulation is done! / Estimation is done!
16: Simulation is done! / Estimation is done!
17: Simulation is done! / Estimation is done!
18: Simulation is done! / Estimation is done!
19: Simulation is done! / Estimation is done!
20: Simulation is done! / Estimation is done!
21: Simulation is done! / Estimation is done!
22: Simulation is done! / Estimation is don

In [7]:
recovery_df

,threshold_true,threshold_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate
0,4.892523,4.975069,0.661988,0.697565,3.104441,3.258323,2.480563,2.500034
1,2.749501,2.767321,0.925488,1.000000,1.239807,1.404526,-0.495842,-0.572819
2,3.709330,5.000000,0.744820,0.387596,1.406281,1.512561,2.065822,2.263050
3,2.783982,2.967909,0.547661,0.529389,-0.846340,-0.736142,2.644126,2.708529
4,2.121912,2.219493,0.788153,0.787404,3.424960,3.500000,-2.437942,-2.448528
...,...,...,...,...,...,...,...,...
145,1.241668,3.955103,0.874422,0.100000,0.569668,0.660856,2.010054,3.500000
146,0.810704,0.879483,0.848814,0.845371,-3.337286,-3.393049,-3.431365,-3.500000
147,4.703830,4.465485,0.900541,0.992409,0.892205,0.912452,-2.429102,-2.520547
148,0.852268,0.858781,0.545772,0.551721,2.945226,2.920980,-2.288738,-2.217341


In [8]:
recovery_df.corr()

,threshold_true,threshold_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate
threshold_true,1.000000,0.886329,0.092468,0.124275,0.137245,0.156889,0.110795,0.082469
threshold_estimate,0.886329,1.000000,0.098692,-0.080437,0.162428,0.177245,0.099026,0.130842
ndt_true,0.092468,0.098692,1.000000,0.761882,-0.064025,-0.064962,-0.015168,0.003147
ndt_estimate,0.124275,-0.080437,0.761882,1.000000,-0.052310,-0.062014,-0.014215,-0.000777
mu1_true,0.137245,0.162428,-0.064025,-0.052310,1.000000,0.984332,0.112457,0.126514
mu1_estimate,0.156889,0.177245,-0.064962,-0.062014,0.984332,1.000000,0.108413,0.118968
mu2_true,0.110795,0.099026,-0.015168,-0.014215,0.112457,0.108413,1.000000,0.956599
mu2_estimate,0.082469,0.130842,0.003147,-0.000777,0.126514,0.118968,0.956599,1.000000


In [9]:
recovery_df.to_csv('Series_2d_recovery_{}.csv'.format(N_series))